### 2 $\alpha$ model
- Estimate learning parameter for chosen and unchosen arms/choices. If the animals are learning, then one should expect that the chosen and unchosen arm will have postive and negative $\alpha$ respectively.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab)
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "beta"],
            "param_values": np.array([qlearn.alpha_c, qlearn.alpha_u, qlearn.beta]),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_params_anirudh")

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns

params_df = mab_subjects.GroupData().qlearning_2alpha_params_anirudh

fig = plotting.Fig(1, 2, size=(4, 3), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp")
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)


alpha_df = params_df[params_df["param"] != "beta"]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

beta_df = params_df[params_df["param"] == "beta"]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### 2 $\alpha$ model + Perserverance + Scaler 
- When we estimated alpha parameters for chosen and unchosen choices, we found that unstructured env had higher alpha values for chosen arms compared to structured env. So we asked if 'persevrance' for arms/choices is making alpha_chosen higher for unstructured environment.

In [27]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps[:1]):
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    qlearn = QlearningEstimator(mab, model="persev", n_cpu=4)
    qlearn.fit(
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]), n_optimize=5
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(params_df, "qlearning_2alpha_persev")

BewilderbeastExp1Structured
alpha_c: 0.0, alpha_u: 0.0,alpha_h: 0.0, scaler: 0.0, beta: 0.0
qlearning_2alpha_persev saved


In [ ]:
import matplotlib.pyplot as plt

# choices = np.array([1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 1])
choices = np.array([1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2])
alpha_h = 0.2
H = 0
h_values = []
for choice in choices:
    H += alpha_h * (choice - H)
    h_values.append(H.copy())


plt.plot(h_values)
plt.plot(choices)

In [ ]:
import numpy as np


class HabitualController:
    def __init__(self, num_actions=2, alpha_H=0.1):
        self.num_actions = num_actions
        self.alpha_H = alpha_H
        self.H = np.zeros(num_actions)  # H0 initialized to zero

    def softmax(self, x, tau=0.1):
        """Softmax action selection with temperature tau (lower tau -> greedier)"""
        exp_x = np.exp(x / tau)
        return exp_x / np.sum(exp_x)

    def select_action(self):
        """Select action based on current habit strengths using softmax"""
        probs = self.softmax(self.H)
        action = np.random.choice(self.num_actions, p=probs)
        return action, probs

    def update(self, action):
        """Update habit strengths based on selected action"""
        a_t = np.zeros(self.num_actions)
        a_t[action] = 1
        self.H += self.alpha_H * (a_t - self.H)


# Example usage (simulating 100 trials)
if __name__ == "__main__":
    np.random.seed(42)
    controller = HabitualController(num_actions=2, alpha_H=0.1)

    for trial in range(100):
        action, probs = controller.select_action()
        print(action)
        controller.update(action)

        print(
            f"Trial {trial+1}: Action={action}, Action Probs={probs}, H={controller.H}"
        )

In [23]:
import numpy as np
from scipy.optimize import minimize


class HabitualControllerMLE:
    def __init__(self, num_actions=2, tau=0.1):
        self.num_actions = num_actions
        self.tau = tau  # softmax temperature

    def softmax(self, x):
        exp_x = np.exp(x / self.tau)
        return exp_x / np.sum(exp_x)

    def compute_neg_log_likelihood(self, alpha_H, actions):
        H = np.zeros(self.num_actions)
        neg_log_likelihood = 0.0

        for action in actions:
            probs = self.softmax(H)
            prob_a = probs[action]
            # Avoid log(0)
            prob_a = np.clip(prob_a, 1e-10, 1.0)
            neg_log_likelihood -= np.log(prob_a)

            # Update H
            a_t = np.zeros(self.num_actions)
            a_t[action] = 1
            H += alpha_H * (a_t - H)

        return neg_log_likelihood

    def fit(self, actions, bounds=(0.001, 1.0)):
        """Estimate alpha_H via Maximum Likelihood given action sequence"""
        result = minimize(
            fun=lambda alpha: self.compute_neg_log_likelihood(alpha[0], actions),
            x0=[0.1],  # initial guess
            bounds=[bounds],
            method="L-BFGS-B",
        )
        estimated_alpha = result.x[0]
        return estimated_alpha, result


# Example usage
if __name__ == "__main__":
    np.random.seed(42)

    # Simulate some actions using true alpha_H=0.2
    true_alpha_H = 0.2
    num_trials = 100
    controller = HabitualControllerMLE()
    H = np.zeros(2)
    actions = []

    for _ in range(num_trials):
        probs = controller.softmax(H)
        action = np.random.choice(2, p=probs)
        actions.append(action)

        a_t = np.zeros(2)
        a_t[action] = 1
        H += true_alpha_H * (a_t - H)

    # Now fit model to recover alpha_H from actions
    model = HabitualControllerMLE()
    est_alpha, result = model.fit(actions)

    print(f"True alpha_H = {true_alpha_H}")
    print(f"Estimated alpha_H = {est_alpha}")

True alpha_H = 0.2
Estimated alpha_H = 0.11682048621725105


In [25]:
import numpy as np
from scipy.optimize import minimize


class HabitualControllerMLE:
    def __init__(self, num_actions=2, tau=0.1):
        self.num_actions = num_actions
        self.tau = tau

    def softmax(self, x):
        exp_x = np.exp(x / self.tau)
        return exp_x / np.sum(exp_x)

    def compute_neg_log_likelihood(self, alpha_H, actions):
        H = np.zeros(self.num_actions)
        neg_log_likelihood = 0.0

        for action in actions:
            probs = self.softmax(H)
            prob_a = probs[action]
            prob_a = np.clip(prob_a, 1e-10, 1.0)
            neg_log_likelihood -= np.log(prob_a)

            a_t = np.zeros(self.num_actions)
            a_t[action] = 1
            H += alpha_H * (a_t - H)

        return neg_log_likelihood

    def fit(self, actions, bounds=(0.001, 1.0)):
        result = minimize(
            fun=lambda alpha: self.compute_neg_log_likelihood(alpha[0], actions),
            x0=[0.1],
            bounds=[bounds],
            method="L-BFGS-B",
        )
        estimated_alpha = result.x[0]
        return estimated_alpha, result


# Your action sequence (1 and 2 -> convert to 0 and 1)
actions = np.array([1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2]) - 1

model = HabitualControllerMLE()
est_alpha, result = model.fit(actions)

print(f"Estimated alpha_H = {est_alpha:.4f}")

Estimated alpha_H = 0.0172


In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

params_df = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="grp", hue_order=["unstruc", "struc"])
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

orders = ["alpha_chosen", "alpha_unchosen", "persev"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

orders = ["scaler", "beta"]
pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
annotator.apply_and_annotate()
annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### Asses estimated params from 2alpha+presev+scaler model

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects
from banditpy.core import MultiArmedBandit

exps = mab_subjects.struc.first_exposure + mab_subjects.unstruc.first_exposure
est_params_df = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh

# param_order = ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"]

perf_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    name = exp.sub_name
    print(name)

    qlearn = QlearningEstimator(mab, bounds=None, model="persev")

    df = est_params_df[est_params_df["name"] == name]
    params = np.array(
        [
            df.loc[df["param"] == "alpha_chosen", "param_values"].to_numpy(),
            df.loc[df["param"] == "alpha_unchosen", "param_values"].to_numpy(),
            df.loc[df["param"] == "persev", "param_values"].to_numpy(),
            df.loc[df["param"] == "scaler", "param_values"].to_numpy(),
            df.loc[df["param"] == "beta", "param_values"].to_numpy(),
        ]
    )
    predicted_choices = qlearn.predict_choices(params=params, deterministic=False)
    predicted_choices[predicted_choices == 0] = 2
    predicted_choices[predicted_choices == 1] = 1

    new_mab = MultiArmedBandit(
        probs=mab.probs,
        choices=predicted_choices,
        rewards=mab.rewards,
        session_ids=mab.session_ids,
        starts=mab.starts,
        stops=mab.stops,
        datetime=mab.datetime,
    )

    sub_df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "trial_id": np.arange(100) + 1,
            "perf": mab.get_performance(),
            "perf_new": new_mab.get_performance(),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    perf_df.append(sub_df)

perf_df = pd.concat(perf_df, ignore_index=True)

mab_subjects.GroupData().save(perf_df, "perf_qlearning_assess_params")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(8, 4, size=(8.5, 11), num=2)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_qlearning_assess_params


ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)
hue_order = ["unstruc", "struc"]
plot_kw = dict(data=df, x="trial_id", y="perf", hue="grp", hue_order=hue_order, ax=ax)
sns.lineplot(
    palette=["#f77189", "#36ada4"],
    # edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)
plot_kw = dict(
    data=df, x="trial_id", y="perf_new", hue="grp", hue_order=hue_order, ax=ax
)
sns.lineplot(
    palette=["#f77189", "#36ada4"],
    linestyle="--",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)

# orders = ["unstruc", "struc"]
# pairs = [(("unstruc"), ("struc"))]
# annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
# annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()
# ax.grid(True)
ax.set_title("Performance")
ax.set_ylabel("Performance")
ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100])
ax.get_legend().remove()

### Params estimation for "Structured" sessions in Unstructured Environment.
- Basically select sessions where the probabilities sum up to 1 i.e, correlated sessions withing Unstructured environment. Estimate various params like alpha_chosen, alpha_unchosen etc. on these sessions and compare these with Structured Environment.

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects

exps = mab_subjects.unstruc.allsess

params_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    session_prob_sum = mab.probs[mab.is_session_start.astype(bool)].sum(axis=1)
    good_sessions = mab.sessions[session_prob_sum == 1]
    assert np.all(good_sessions)

    mab = mab.keep_sessions_by_id(good_sessions)

    qlearn = QlearningEstimator(mab, model="persev")
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0, 1), (1, 10), (0.005, 20)]),
        method="diff_evolution",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "param": ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"],
            "param_values": qlearn.estimated_params,
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)
mab_subjects.GroupData().save(
    params_df, "qlearning_2alpha_persev_correlated_within_unstructured_anirudh"
)

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
from statplot_utils import stat_kw

df1 = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh
df2 = (
    mab_subjects.GroupData().qlearning_2alpha_persev_correlated_within_unstructured_anirudh
)
df2["grp"] = "struc_in_unstruc"
params_df = pd.concat([df1, df2], ignore_index=True)

fig = plotting.Fig(1, 3, size=(6, 4), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(
    x="param",
    y="param_values",
    hue="grp",
    hue_order=["unstruc", "struc", "struc_in_unstruc"],
)
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)

indx_bool = params_df["param"].isin(["scaler", "beta"])

alpha_df = params_df[~indx_bool]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

# orders = ["alpha_chosen", "alpha_unchosen", "persev"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=alpha_df, ax=ax1, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


beta_df = params_df[indx_bool]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

# orders = ["scaler", "beta"]
# pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
# annotator = Annotator(pairs=pairs, data=beta_df, ax=ax2, **plot_kw, order=orders)
# annotator.configure(test="t-test_ind", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()


ax1.axhline(0, ls="--", color="gray", zorder=0, lw=0.8)
ax2.set_xlim(-1, 2)
ax2.set_ylim(1, 15)

ax1.tick_params(axis="x", rotation=30)
ax2.tick_params(axis="x", rotation=30)

ax1.set_xlabel("")
ax2.set_xlabel("")

ax1.legend_.remove()
ax2.legend_.remove()
ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

### Fit Qlerning params from one env to other.
- Take average of parameters like alpha_chosen, persev etc. from structured environment and predict actions in unstructured env. Following this calculate performance and compare that to actual performance. 

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import QlearningEstimator
import mab_subjects
from banditpy.core import MultiArmedBandit

exps = mab_subjects.struc.first_exposure + mab_subjects.unstruc.first_exposure
est_params_df = mab_subjects.GroupData().qlearning_2alpha_persev_anirudh
mean_params_df = (
    est_params_df.groupby(["grp", "param"]).mean(numeric_only=True).reset_index()
)

mean_struc = mean_params_df[mean_params_df["grp"] == "struc"]
mean_unstruc = mean_params_df[mean_params_df["grp"] == "unstruc"]
param_order = ["alpha_chosen", "alpha_unchosen", "persev", "scaler", "beta"]

perf_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.filter_by_trials(min_trials=100, clip_max=100)
    name = exp.sub_name
    print(name)

    qlearn = QlearningEstimator(mab, bounds=None, model="persev")

    if mab.is_structured:
        df = mean_unstruc.copy()
    else:
        df = mean_struc.copy()

    params = np.array(
        [
            df.loc[df["param"] == "alpha_chosen", "param_values"].to_numpy(),
            df.loc[df["param"] == "alpha_unchosen", "param_values"].to_numpy(),
            df.loc[df["param"] == "persev", "param_values"].to_numpy(),
            df.loc[df["param"] == "scaler", "param_values"].to_numpy(),
            df.loc[df["param"] == "beta", "param_values"].to_numpy(),
        ]
    )
    predicted_choices = qlearn.predict_choices(params=params, deterministic=False)
    predicted_choices[predicted_choices == 0] = 2
    predicted_choices[predicted_choices == 1] = 1

    new_mab = MultiArmedBandit(
        probs=mab.probs,
        choices=predicted_choices,
        rewards=mab.rewards,
        session_ids=mab.session_ids,
        starts=mab.starts,
        stops=mab.stops,
        datetime=mab.datetime,
    )

    sub_df = pd.DataFrame(
        {
            "name": exp.sub_name,
            "trial_id": np.arange(100) + 1,
            "perf": mab.get_performance(),
            "perf_new": new_mab.get_performance(),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    perf_df.append(sub_df)

perf_df = pd.concat(perf_df, ignore_index=True)

mab_subjects.GroupData().save(perf_df, "perf_qlearning_switch_params")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(8, 4, size=(8.5, 11), num=1)

grpdata = mab_subjects.GroupData()
df = grpdata.perf_qlearning_switch_params


ax = fig.subplot(fig.gs[0])
# ax.axhline(0, color="gray", lw=0.8, zorder=0)
hue_order = ["unstruc", "struc"]
plot_kw = dict(data=df, x="trial_id", y="perf", hue="grp", hue_order=hue_order, ax=ax)
sns.lineplot(
    palette=["#f77189", "#36ada4"],
    # edgecolor="white",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)
plot_kw = dict(
    data=df, x="trial_id", y="perf_new", hue="grp", hue_order=hue_order, ax=ax
)
sns.lineplot(
    palette=["#f77189", "#36ada4"],
    linestyle="--",
    # facecolor=(0, 0, 0, 0),
    # alpha=0.4,
    err_kws=dict(edgecolor="none"),
    errorbar="se",
    **plot_kw,
)

# orders = ["unstruc", "struc"]
# pairs = [(("unstruc"), ("struc"))]
# annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
# annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
# annotator.apply_and_annotate()
# annotator.reset_configuration()
# ax.grid(True)
ax.set_title("Performance")
ax.set_ylabel("Performance")
ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100])
ax.get_legend().remove()